# Middleware: Human In The Loop
<img src="./assets/LC_HITL.png" width="300">



## Setup

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community langgraph databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

In [ ]:
# No Databricks, usamos spark.sql() diretamente
# Verifique as tabelas disponíveis no catálogo NASA GCN
%sql
SHOW TABLES IN nasa_gcn.silver

In [ ]:
# RuntimeContext não é necessário quando usamos spark diretamente

In [ ]:
from langchain_core.tools import tool


@tool
def execute_sql(query: str) -> str:
    """Execute a SQL command on Databricks and return results."""
    try:
        result = spark.sql(query).limit(5).toPandas()
        return result.to_markdown()
    except Exception as e:
        return f"Error: {e}"

In [ ]:
SYSTEM_PROMPT = """Você é um analista cuidadoso de Databricks SQL especializado em dados da NASA GCN.

Regras:
- Pense passo a passo.
- Quando precisar de dados, chame a ferramenta `execute_sql` com UMA query SELECT.
- Apenas leitura; sem INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limite a 5 linhas de saída, a menos que o usuário peça mais.
- Se a ferramenta retornar 'Error:', revise o SQL e tente novamente.
- Prefira listas explícitas de colunas; evite SELECT *.
- Use o catálogo nasa_gcn e schema silver (nasa_gcn.silver.tabela).
- Se o banco de dados estiver offline, peça ao usuário para tentar novamente mais tarde.
"""

## Middleware

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"execute_sql": {"allowed_decisions": ["approve", "reject"]}},
        ),
    ],
)

In [ ]:
from langgraph.types import Command

question = "Liste todos os eventos do catálogo nasa_gcn schema silver"

config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
)

if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )

    # Simulando REJEIÇÃO - database offline
    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "reject", "message": "o banco de dados está offline."}]
            }
        ),
        config=config,  # Same thread ID to resume the paused conversation
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

In [ ]:
# Agora com APROVAÇÃO automática
question = "Quais são os tipos de eventos mais frequentes no nasa_gcn.silver?"

config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
)

while "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )
    
    # Aprovando a execução
    result = agent.invoke(
        Command(
            resume={"decisions": [{"type": "approve"}]}
        ),
        config=config,  # Same thread ID to resume the paused conversation
    )

for msg in result["messages"]:
    msg.pretty_print()